In [2]:
!pip install -q tensorflow==2.0.0-beta1
!pip install python-chess

     |████████████████████████████████| 87.9MB 482kB/s 
     |████████████████████████████████| 3.1MB 27.3MB/s 
     |████████████████████████████████| 501kB 43.5MB/s 


In [3]:
import numpy as np
import tensorflow as tf
import chess

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, Lambda
from keras import backend as K
from tensorflow import keras

Using TensorFlow backend.


In [4]:
mat = np.random.randint(0, 100, (3, 6, 5))
mat

array([[[65, 74, 24, 49, 48],
        [40, 53, 35, 79, 72],
        [88, 19, 59, 62,  9],
        [72, 35, 42, 66,  8],
        [95, 33, 15, 10, 92],
        [53, 79, 45, 23, 59]],

       [[53, 55, 47, 13, 46],
        [ 4, 94, 97, 34, 25],
        [ 7, 63, 92, 83, 78],
        [22, 11, 25, 59, 73],
        [24, 12, 30, 41, 56],
        [54, 73, 26, 35, 24]],

       [[71, 76, 53, 46, 92],
        [28, 70, 84, 59, 56],
        [89, 60, 30, 91, 94],
        [36, 22, 38, 85, 20],
        [37, 14, 98,  5, 26],
        [75, 57, 91, 48, 96]]])

In [5]:
board = chess.Board()
board.legal_moves

<LegalMoveGenerator at 0x7f1c13d1fda0 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>

In [0]:

def get_mat(piece, color):
  squareSet = board.pieces(piece, color)
  return get_bin_mat(squareSet);

def get_bin_mat(squareSet):
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in squareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
            
  return mat


def get_dual_bin_mat(piece):
  whiteSquareSet = board.pieces(piece, chess.WHITE)
  blackSquareSet = board.pieces(piece, chess.BLACK)
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in whiteSquareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
        elif(y*8 + x in blackSquareSet):
          mat[7 - y, x] = -1
            
  return mat

Input data:

  (6 pieces + 1 for meta state ) * 2 players
  
  8 rows
  
  8 columns
  
 => 8 * 8 * 12 = 768
 
 
 Output data:
 
 - Assume predicting five moves
 - initial location, and new location
 
 
Array of objects/tuples
[{initial location, new location},...]
 
 
 => 5 * 2 * 64 = 640
  

In [7]:
mat = get_dual_bin_mat(chess.PAWN)
mat.shape

test = np.array([get_mat(i, chess.WHITE) for i in range(1,7)] + [get_mat(i, chess.BLACK) for i in range(1,7)])
test.shape

y = test.reshape((-1, 12, 8, 8))
y.shape

(1, 12, 8, 8)

In [0]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(12,8,8)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='linear'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
x = model.predict(y)
x

array([[ 0.04705101, -0.02663097, -0.04413891,  0.01072281,  0.04222774,
         0.01144807, -0.01855633, -0.02139407, -0.03520958,  0.04532561]],
      dtype=float32)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 10, 6, 64)         4672      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 32)          18464     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                10250     
Total params: 33,386
Trainable params: 33,386
Non-trainable params: 0
_________________________________________________________________


In [24]:
NUM_OUTPUTS = 5


#inp is a "tensor", that can be passed when calling other layers to produce an output 
inp = Input((12,8,8)) #supposing you have ten numeric values as input 


#here, SomeLayer() is defining a layer, 
#and calling it with (inp) produces the output tensor x
# x = SomeLayer(blablabla)(inp) 
# x = SomeOtherLayer(blablabla)(x) #here, I just replace x, because this intermediate output is not interesting to keep

x = Conv2D(64, kernel_size=3, activation='relu', input_shape=(12,8,8))(inp)

x = Conv2D(32, kernel_size=3, activation='relu', input_shape=(12,8,8))(x)

x = Flatten()(x)


outputs = []

for n in range(NUM_OUTPUTS):
  outputs.append(Dense(128, activation='softmax')(x))


#here, I want to keep the two different outputs for defining the model
#notice that both left and right are called with the same input x, creating a fork
# out1 = LeftSideLastLayer(balbalba)(x)    
# out2 = RightSideLastLayer(banblabala)(x)


#here, you define which path you will follow in the graph you've drawn with layers
#notice the two outputs passed in a list, telling the model I want it to have two outputs.
model = Model(inp, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #loss can be one for both sides or a list with different loss functions for out1 and out2    

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 12, 8, 8)]   0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 10, 6, 64)    4672        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 8, 4, 32)     18464       conv2d_4[0][0]                   
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 1024)         0           conv2d_5[0][0]                   
____________________________________________________________________________________________

In [0]:
p = model.predict(y)
# np.argmax(p[0])


In [0]:
def interpret_pred(p):
  num_outputs = len(p)
  
  moves = []
  for out in p:
    # There's an extra list wrapping for some reason
    out = out[0]
    start = np.argmax(out[:64])
    end = np.argmax(out[64:])
    
    #print(chess.SQUARE_NAMES[start])
#     print(end)
    
    moves.append((chess.SQUARE_NAMES[start],
                  chess.SQUARE_NAMES[end]))
    
  return moves

In [52]:
moves = interpret_pred(p)
print(moves)

[('f1', 'b4'), ('b4', 'a5'), ('f4', 'g3'), ('h6', 'd7'), ('g4', 'c5')]


In [39]:
p

[array([[0.00764698, 0.00706254, 0.00771121, 0.00779736, 0.00793065,
         0.00873157, 0.00770415, 0.00780445, 0.0075901 , 0.00765776,
         0.00835187, 0.00739352, 0.00766488, 0.00761447, 0.00710705,
         0.00758018, 0.00755843, 0.00779666, 0.00755307, 0.00769361,
         0.00822804, 0.00709443, 0.00801772, 0.00816545, 0.00871106,
         0.00732566, 0.00795426, 0.00860787, 0.00828659, 0.00798255,
         0.00854007, 0.00810956, 0.00793171, 0.00761743, 0.00775256,
         0.00785392, 0.00804098, 0.00725975, 0.00787752, 0.00778343,
         0.00724136, 0.00689761, 0.00785535, 0.00748082, 0.00816488,
         0.00748195, 0.00778243, 0.00826092, 0.00753153, 0.00799314,
         0.00841984, 0.00808385, 0.0083874 , 0.00839574, 0.00805726,
         0.00759641, 0.00766094, 0.00800855, 0.00788796, 0.00834842,
         0.00729032, 0.00727751, 0.00831852, 0.00757592, 0.00795694,
         0.00733263, 0.00724402, 0.00805666, 0.00726766, 0.00721852,
         0.00798206, 0.00856476, 0